# DEM, Shapefile import and projection
A notebook going through different ways to import and project shape and DEM files.

In [ ]:
import rioxarray
import xarray
import geopandas
import shapely.geometry
import rasterio
import rasterio.mask
import numpy
import matplotlib.pyplot

## Load in data
Note that the LiDAR (and DEMs were downloaded from OpenTopgraphy with EPSG:2193)
Horizontal Coordinates: NZGD2000 / New Zealand Transverse Mercator 2000 [EPSG: 2193]
Vertical Coordinates: NZVD2016 [EPSG: 7839]

In [ ]:
# from Open Topography
dem_tiff=rioxarray.rioxarray.open_rasterio(r'C:\Users\pearsonra\Documents\data\herbert_10res\dem\dem.tif', masked=True)
dem_nc=rioxarray.rioxarray.open_rasterio(r'C:\Users\pearsonra\Documents\data\herbert_10res\dem\dem.nc', masked=True)
# from QGIS
ocean_shape = geopandas.read_file(r'C:\Users\pearsonra\Documents\data\herbert_10res\fake_coast.shp')
ocean_shape_projected = geopandas.read_file(r'C:\Users\pearsonra\Documents\data\herbert_10res\ocean_shape_projected.shp')

Note that net_CDF files can also be opened directly by xarray retaining more information and keeping its initial form (i.e. not adding bands and an extra dimension)

In [ ]:
xarray.open_dataset(r'C:\Users\pearsonra\Documents\data\herbert_10res\dem\dem.nc')

## View and correct projections
### Projection for DEMs

In [ ]:
dem_tiff.rio.crs

In [ ]:
dem_nc.rio.crs

#### Correct netCDF crs 
note the correction is needed due to export setting incorrectly being selected from QGIS - skip this stage in future and just bring into xarray (same as nextCDF anyway)

In [ ]:
dem_nc=dem_nc.rio.set_crs(dem_tiff.rio.crs)

### Projection from Shape files

In [ ]:
ocean_shape.crs

In [ ]:
ocean_shape_projected.crs

#### correct unprojected shapefile crs 
reproject the QGIS polygon

In [ ]:
ocean_shape = ocean_shape.to_crs(dem_tiff.rio.crs)

## View DEM and select region to filter
Below the dem and the ocean shape file - saved out of QGIS with a projection.

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 5))
dem_tiff.plot(cmap="Greys_r",
                 ax=ax)
ocean_shape.plot(color='None',
                    edgecolor='teal',
                    linewidth=2,
                    ax=ax,
                    zorder=4)
ax.set_title("LiDAR Herbert - with coast projection from QGIS")
#ax.set_axis_off()
matplotlib.pyplot.show()

The DEM zoomed in on the region to manually create a polygon here

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 5))

dem_tiff.plot(cmap="Greys_r", ax=ax)

ax.set_title("LiDAR Herbert - zoomed in on coast")
matplotlib.pyplot.xlim([1.43222e6, 1.434e6])
matplotlib.pyplot.ylim([4.9857e6, 4.9873e6])

matplotlib.pyplot.show()

## Manually create a polygon

In [ ]:
ocean_manual_shape = shapely.geometry.Polygon([(1.43222e6, 4.9857e6), # x1, y1
            (1.43400e6, 4.9857e6), # x2, y1
            (1.43400e6, 4.9873e6), # x2, y2
            (1.43222e6, 4.9873e6)  # x1, y2
        ])
ocean_manual_gdf = geopandas.GeoSeries([ocean_manual_shape])

In [ ]:
# Plot the data and notice that the scale bar looks better
# No data values are now masked
f, ax = matplotlib.pyplot.subplots(figsize=(10, 5))

ocean_manual_gdf.plot(color='None', edgecolor='teal', linewidth=20, ax=ax, zorder=4)

ax.set_title("Manual polygon \n selects the region out to coast")
matplotlib.pyplot.show()

## Plot both together

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(10, 5))

dem_tiff.plot(cmap="Greys_r", ax=ax)
ocean_manual_gdf.plot(color='None', edgecolor='teal', linewidth=2, ax=ax, zorder=4)

ax.set_title("LiDAR Herbert")
matplotlib.pyplot.show()